In [12]:
import pandas as pd
import numpy as np
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier #KNN 알고리즘 모델 클래스
from sklearn.tree import DecisionTreeClassifier # Tree 알고리즘 모델 클래스
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
import mysql.connector 
from flask import Flask, jsonify, request

In [4]:
diabetes = pd.read_csv("./diabetes (1).csv")
hypertension = pd.read_csv("./hypertension_data.csv")
stroke = pd.read_csv("./stroke_data.csv")


In [5]:
def connect_to_mysql():
    db_config = {
        'host':'project-db-campus.smhrd.com',
        'port':3307,
        'user':'yeaha',
        'password':'1234',
        'database':'yeaha'
    }
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    return conn,cursor

def close_mysql_connection(conn, cursor):
    cursor.close()
    conn.close()

def execute_query(conn, cursor, query):
    try:
        cursor.execute(query)
        columns = [column[0] for column in cursor.description]
        results = []
        for row in cursor.fetchall():
            result_dict = dict(zip(columns, row))
            results.append(result_dict)
        return results
    except mysql.connector.Error as err:
        print(f"Error: {err}")

In [6]:
conn, cursor = connect_to_mysql()

try:
    query = "select * from tbl_product limit 1"
    result = execute_query(conn, cursor, query)
finally:
    close_mysql_connection(conn, cursor)

In [ ]:

from openai import OpenAI

client = OpenAI(api_key="__")

text = '칼슘'
completion = client.chat.completions.create(model='gpt-3.5-turbo', messages=[{"role": "user", "content": f"{text}이랑 먹으면 안좋은 영양소 한개만 말해줘 대답하지말고 한단어만 출력해줘"}])

completion.choices[0].message.content

'철분'

In [ ]:
# 건강검진데이터, 고혈압데이터, 당뇨데이터,  뇌졸중데이터, 심장병데이터, 유저건강검진데이터
def pre_model_data(user_data):
    import pandas as pd
    #from sklearn.model_selection import train_test_split
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.neighbors import KNeighborsClassifier #KNN 알고리즘 모델 클래스
    from sklearn.metrics import accuracy_score
    
    model_test1 = KNeighborsClassifier(n_neighbors=10)
    model2 = KNeighborsClassifier(n_neighbors=40)
    model3 = KNeighborsClassifier(n_neighbors=15)
    model_heart = KNeighborsClassifier(n_neighbors=23)
    
    
    file_diabetes = "C:/Users/smhrd/3.AI(ML, DL)/merge_data1/diabetes.csv"
    df_die = pd.read_csv(file_diabetes)
    
    file_hypertension = "C:/Users/smhrd/3.AI(ML, DL)/merge_data1/hypertension_data.csv"
    hypertension = pd.read_csv(file_hypertension)
    
    file_stroke = "C:/Users/smhrd/3.AI(ML, DL)/merge_data1/stroke_data.csv"
    df_stroke = pd.read_csv(file_stroke)
    
    file_heal = "C:/Users/smhrd/3.AI(ML, DL)/merge_data1/국민건강보험공단_건강검진정보_20221231 (1).CSV"
    df_heal = pd.read_csv(file_heal, encoding='euc-kr')
    
    file_heart = "C:/Users/smhrd/3.AI(ML, DL)/merge_data1/heart_failure_clinical_records_dataset.csv"
    df_heart = pd.read_csv(file_heart, encoding='euc-kr')
    
    df_sl = df_heal[["가입자일련번호", '성별', '연령대코드(5세 단위)', '신장(5cm단위)', 
                     '체중(5kg단위)', '허리둘레', '시력(좌)', '시력(좌)', '시력(우)', '청력(좌)', 
                     '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '요단백', '흡연상태', 
                     '음주여부', '총콜레스테롤']]
    
    df_sl['bmi'] = df_sl['체중(5kg단위)']/ (df_sl['신장(5cm단위)']/100 * df_sl['신장(5cm단위)']/100)
    df_sl['bmi'] = df_sl['bmi'].round(1)
    
    df = df_sl.dropna()
    
    df_die1 = df_die[['Glucose', 'Outcome', 'BMI']]
    df_die2 = df[['식전혈당(공복혈당)']]
    df_hyper1 = df_hyper.dropna()
    
    X2 = df_hyper1[['age', 'sex', 'trestbps', 'chol']]
    y = df_hyper1['target']
    X1 = df[['성별', '연령대코드(5세 단위)', '이완기혈압', '총콜레스테롤']]
    X1.columns = ['age', 'sex', 'trestbps', 'chol']
    X3 = X1[['age', 'sex', 'trestbps', 'chol']]
    
    X_train, X_test, y_train, y_test = train_test_split(X2, y, random_state=133)
    
    
    model_test1.fit(X_train, y_train)
    
    pre_hyper = model_test1.predict(X3)
    
    df_die2.columns = ['Glucose']
    
    X = df_die1[['Glucose']]
    y = df_die1['Outcome']
    
    model2.fit(X,y)
    
    
    pre_die = model2.predict(df_die2)
    
    df_stroke = df_stroke.dropna()
    
    df['성별-1'] = df['성별'] - 1
    df['흡연상태-1'] = df['흡연상태']-1
    df['hypertension'] = pre_hyper
    
    X = df_stroke[['hypertension', 'bmi', 'smoking_status']]
    y = df_stroke['stroke']
    df111 = df[['hypertension', 'bmi', '흡연상태-1']]
    df111.columns = ['hypertension', 'bmi', 'smoking_status']
    
    model3.fit(X,y)
    
    pre_stroke = model3.predict(df111)
    
    df["diabetes"] = pre_die
    
    df_heart1 = df_heart[['age', 'high_blood_pressure', 'smoking']]
    df_sl123 = df[['연령대코드(5세 단위)', 'hypertension', '흡연상태-1']]
    df_sl123.columns = ['age', 'high_blood_pressure', 'smoking_status']
    df_heart1.rename(columns={'smoking': 'smoking_status'}, inplace=True)
    df_heart1.rename(columns={'high_blood_pressure': 'hypertension'}, inplace=True)
    
    X = df_heart1
    y = df_heart["DEATH_EVENT"]
    
    X_heart_train, X_heart_test, y_heart_train, y_heart_test = train_test_split(X,y, random_state=110)
    
    model_heart.fit(X_heart_train, y_heart_train)
    
    user_hyper = model_test1.predict(user_data[['age', 'sex', 'trestbps', 'chol']])
    user_data['hypertension'] = user_hyper
    
    user_die = model2.predict(user_data[['Glucose']])
    
    user_stoke = model3.predict(user_data[['hypertension', 'bmi', 'smoking_status']])
    
    user_heart = model_heart.predict(user_data[['age', 'hypertension', 'smoking_status']])
    
    data = {"hypertension" : user_hyper, "diabetes" : user_die, "stoke" : user_stoke, "heart" : user_heart}
    series = pd.Series(data)
    
    return series

In [10]:
def recommendAlgorithm(series):
    
    
    recommend_list = []
    
    # 'hyper_pre1', 'dia_pre1', 'stroke', 'heart'
    test_dis_list = ['고혈압', '당뇨', '뇌졸중', '심장병']
    
    # 받아온 시리즈 데이터 1 인 자리의 질병이름만 추출
    selected_diseases = [disease for disease, value in zip(test_dis_list, series) if value == 1]
    
    # 질병명을 통해 필요한 영양소가 무엇인지 추출
    conn, cursor = connect_to_mysql()
    
    dis_name_list = []
    # 질병명과 영양소 가져오기
    # 재사용할땐 질병명이 필요할수도있으니..
    for d in selected_diseases:
        query = f"select 질병명, 영양소 from tbl_mentor_disease where 질병명 LIKE '%{d}%'"
        res = execute_query(conn, cursor, query)
        dis_name_list.extend(res)
    
    # 영양소만 분리
    nutrient_list = []
    for j in dis_name_list:
        nutrient_list.append(j['영양소'].replace(" ", ''))
    
    # 제약조건 chat-GPT 연동 통해서 추가
    from openai import OpenAI
    client = OpenAI(api_key="__")
    
    
    # 건강기능식품추천
    index=0
    for k in nutrient_list:
        text = k
        completion = client.chat.completions.create(model='gpt-3.5-turbo', messages=[{"role": "user", "content": f"{text}이랑 먹으면 안좋은 영양소 한개만 말해줘 대답하지말고 한단어만 출력해줘"}])
        avoid = completion.choices[0].message.content
        query = f"select prdlst_nm from tbl_product2 where rawmtrl_nm like '%{k}%' and rawmtrl_nm not like '%{avoid}%'"
        res = execute_query(conn, cursor, query)
        recommend_list.extend(res)

    close_mysql_connection(conn,cursor)
    
    if len(recommend_list) == 0:
        return "오류 발생으로 다시 테스트해주세요"
    else:
        values = [item['prdlst_nm'] for item in recommend_list]
        return list(set(values))[0:10]

In [ ]:
recommendAlgorithm([0, 1, 1 ,0])

['해조칼슘비타민',
 '어덜트멀티비타민&미네랄',
 '고려홍삼골드플러스',
 '그린키즈츄어블아연딸기맛',
 '고려홍삼정(錠)',
 '고삼력홍삼캡슐골드32',
 '옵티덴골드',
 '모닝큐화이버27',
 '뽀롱뽀롱 뽀로로 멀티비타민',
 '고려인삼정캡슐']

In [ ]:
app = Flask(__name__)

# OpenAI API Key
openai_api_key = "__"
client = OpenAI(api_key=openai_api_key)

@app.route('/')
def home():
    return "환영합니다! 질환 유형을 선택하여 관련 제품을 추천받으세요."
@app.route('/recommend', methods=['GET'])
def recommend():
    try:
        # Get data from request args
        series = [int(request.args.get('val1')), int(request.args.get('val2')), int(request.args.get('val3')), int(request.args.get('val4'))]
        
        recommend_list = recommendAlgorithm(series)
        
        # Return recommended list as JSON
        return jsonify(recommend_list)

    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(port=5005)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5005
Press CTRL+C to quit
127.0.0.1 - - [07/Mar/2024 10:54:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2024 10:54:55] "GET /recommend?val1=1&val2=0&val3=1&val4=0 HTTP/1.1" 200 -
